<a href="https://colab.research.google.com/github/SeongwonTak/RecommenderSystem/blob/main/Fundamentals_of_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#추천 시스템의 개요

한달 전에 유튜브를 통해 추천시스템에 대해서 공부해보았는데, 조금 더 자세히 관련 내용 및 일부 논문을 찾아보며 내용에 대해 보강하고자 한다.

(사실 내용 수정이 어느정도 필요하기에 다시 처음부터 공부하였다..)

참고자료

[1] Toward the Next Generation of Recommender Systems: A survey of the State-of-the-Art and Possible Extensions

[2] Matrix Factorization techniques for recommender Systems (Yehuda Koren, Robert Bell and Chris Volinsky)

[3] https://darkpgmr.tistory.com/106?category=460967 (SVD파트)


크게 추천 시스템은
- **content_based filtering** : 비슷한 아이템에 대해
- **collaborative filtering** : 비슷한 유저의 과거 행동/성향 기반

으로 나눌 수 있다.

그리고 이 둘을 혼합한
**hybrid filtering**이 존재한다.


### formal notations
조금 더 formal하게 쓰자면
- C : user set
- S : items set
- R : rating set (totally ordered..)

그러면 이 경우, 우리가 풀고 싶은 문제는 함수
$$u : C \times S \rightarrow R $$
이 주어질 때, user $c$에 대해, 다음과 같은 $s'$를 찾는것이다.
$$s'_{c} = argmax_{s \in S} u(c,s) $$

하지만 rating이라는 것이, 굉장히 sparse하게 주어질 것이기에, 우리는 확장(extrapolated to the whole space)가 필요하다.

이러한 확장을 위해서는
- specifying heuristics
- estimating the utility ftn. that optimizes permormance criterion.

##content_based filtering
- 사용자가 특정 아이템에 대해 선호할 경우, 그 아이템과 유사한 아이템을 보여주는 전통적인 방식의 추천 모델
- [1] Text-based item에서 많이 사용하게 된다.

### Text-based item recommendations
먼저 기호를 정의한다.
- 문서번호 : $d_{j}$
- 단어번호 : $k_{j}$
- weighting measure $w_{ij}$: 문서 j에 있는 keyword $k_{i}$중요도

weighting measure을 잡는 다양한 방법들이 있는데 [1]에서는 **TF-IDF** 소개.

term frequency/inverse document frequency(TF-IDF)

- $N$ : total num of docs.
- $n_{i}$ : num. of docs that appears of keyword $k_{j}$
- $f_{i,j}$ : num. of keyword k_{i} appears in doc. $d_{j}$

그러면 
- TF : the term freq. of keyword $k_{i}$ in doc. $d_{j}$
$$TF_{i,j}=\frac{f_{i,j}}{max_{z}f_{z,j}} $$

하지만, 이 TF만 사용할 경우에는 실질적으로 문서들간의 차이를 구분지을 수있는 특이한 단어들(즉, 문법적 요소가 아닌 단어들)을 강조하기는 어렵다.

그렇게 도입한 것이 IDF이다

- IDF : The inverse doc. freq. for keyword $k_{i}$
$$ IDF_{i} = log \frac{N}{n_{i}}$$

$w_{i, j} = TF_{i,j} \times IDF_{i}$로 가중치를 얻게 된다. 이를 통해 item의 특성을 얻게 된다.

### similar item 고르기
User별 keyword별로 weight를 부여한다.
즉 User별 keyword의 특징을 얻게 된다.
- Rocchio algorithm을 사용하면 ContentBasedProfile의 평균을 계산하다.
- Bayesian classfier을 활용하여, 특정 아이템을 좋아할 확률을 계산한다.
- feature가 많을 경우에는 Winnow Algorithm을 쓰게 된다.

cosine measure를 통해, 유사성이 높은 다른 아이템을 추천받을 수 있다.
(cosine 값을 구하여, 유사한 방향을 가지고 있다면 유사성이 높을 것이다)

질문) 이 경우, 별점테러범이, 원하지 않는 분야의 아이템만 추천받는..
(물론 특이한 상황이지만. 낮은 평점 위주로 주는 사람이 만족할만한 추천을 받을 수 있는 지표인지 의문이 된다)

### Bayesian classifiers의 활용
User가 좋아할 확률을 추측해 볼 수 있는 방법 중의 하나로 진행.
(Model-based)

Naive Bayes classifier를 활용하여 content-based 모델을 구성할 수 있다.
Web-page인 경우를 예를 들면 Naive Bayes Classifier는 unrated된 page가 유저가 높은 rate를 준 page들과 relevent한가를 판단할 수 있을 것이다.

구체적으로는 다음 확률에 기반을 두게 될 것이다.

페이지 $p_{j}$가 class $C_{i}$에 속할 확률은,
$$P(C_{i}|k_{1,j}, k_{2,j},...,k_{n,j}  ) $$
(특정 단어가 포함되는가로 스팸 필터링 문제와 동일해진다.)

따라서, Naive Bayes를 적용하기 위해 우리는 각 keyword들이 '독립적'이라고 가정할 것이다.


### Limitations
[1]에서 언급한 content based filtering의 한계들

#### feature가 불충분하먼 곤란.
- feature가 불충분할 경우, 두개의 서로 다른 아이템이 같은 feature 집합을 가질 경우 두 아이템을 구분할 수 없다.
- 이를 위해 충분한 feature를 추출해내야 하는데, 그림이나 음성 데이터 등에서는 이에 대한 추출이 아! 매우 어려움!

#### Overspecialization
추천의 다양성을 어떻게 확보할 수 있을지가 관건.

- 유저가 전혀 경험하지, 혹은 평가를 매기지 않은 영역에 대해서는 추천 받기 어렵다.
- 또한, 너무 비슷한(유사한) 아이템을 추천받는 것도 곤란할 수 있다.
뉴스의 예를 들면, 이미 본 내용과 똑같은 내용의 다른 기사를 보는게 의미가 있을까?

#### New User Problem
실제 User의 preference를 알기 위해서는 초기 충분한 양의 rate 평가가 이루어져야 한다. 즉, 극초기에는 충분히 좋은 추천을 받기가 어려울 것이다.
(Cold Start)

##collaborative filtering
- 비슷한 유저의 과거 행동에 의존한다.
  - 즉 과거의 구매 이력이나 평점 이력을 통해 유저들에게 새로운 것을 추천한다.
  - collaborative filtering 방법도 **"Cold Start"** 문제를 겪게 된다. 최초 새로 user 프로필이나 신규 아이템의 경우는 데이터가 없기에 유의미한 추천을 하기 어렵다.
    - 이러한 경우에 대해서는 "Content-based" 모델이 더 낫다.

- Collaborative filtering은 크게 다음 두가지로 나뉜다.
  - neighborhood methods
  - latent factor models

- [1]에서는 다른 방식으로 나누었다.
  - Heuristic-based (Memory-based)
  - Model-based

### neighborhood methods란?
- 매긴 rating을 바탕으로 user들과 비교, 유사한 user를 추천한다.
- row가 user, column이 item인 형태의 행렬을 만들어야 한다.

[1]에서는 **Memory-based model**이라는 명칭을 사용한다.
기본적으로 Unknown한 rating을 추론하기 위해, 이미 다른 유저들이 평가한 rating을 바탕으로 결과값을 얻는다.

이를 위해서는 '이웃'에 대해서 고려해야 한다.


#### Aggregation Functions
간단하게 memory-based model을 식으로 표현하자.

- 풀고자 하는 바

문제를 식으로 정의하면 다음과 같다.
$$ r_{c,s} = aggr_{c\ \in \hat{C}} \; r_{c', s}$$

여기서, $\hat{C}$는 c와 유사한 유저들 중 item $s$에 평점을 매긴 사람들의 집합이다.

즉, 이 문제는 어떻게 aggr을 표현할 것인가와 연관된다. 후보 함수들에는 다음 것들이 있다.

- (a)$r_{c,s} = \frac{1}{N}\sum_{c' \in \hat{C}} r_{c',s} $
- (b)$r_{c,s} = k\sum_{c' \in \hat{C}}sim(c,c') \times r_{c',s}$
- (c)$r_{c,s} = \bar{r_{c}}+k\sum_{c' \in \hat{C}}sim(c,c')\times (r_{c',s}-\bar{r_{c'}})$

여기서 
- $k$는 normalizing factor로, 보통 $k=1 / \sum_{c' \in \hat{C}}|sim(c,c')|$으로 쓰인다.
- $\bar{r_{c}} = (1/|S_{c}|)\sum_{s \in S_{c}}r_{c,s}$ where $S_{c}=\{s\in S | r_{c,s}\neq\emptyset\}$

(a)는 가장 기본적인 평균이다.

그러나 (b)의 경우에는 sim을 활용하여 더 가까운 아이템에 가중치를 주는 형식이다. 그리고 가중치를 곱하기에 normalizing을 실시해야 한다.

(c)의 경우에는 (b)에서 유저들간 평점을 매길때 유저들이 적용하는 scale의 차이를 반영한 개선식이다.

sim의 경우는 cosine 혹은 Pearson similarity를 쓸 수도 있다.

#### Improving models
모델의 발전을 위해 

이와는 별도로, 이 방식을 user기반이 아닌, item기반으로도 고려할 수 있다.

### Model based란?

Memory-based가 user나 item간의 유사성을 기준으로 한다면, Model-based는 통계학적 기법, 머신러닝 기법들을 사용하여 추천을 진행한다는 점이다.

Remark. 둘 중 어느 것이 뛰어나다는 확실한 증거는 없지만 경험적으로(emperical) Model-based의 추천 성능이 더 우수한것으로 알려져 있다고 한다.

#### Example. probabilistic approach

하나의 방법은 확률론적 방법이다.

이는 기대값에 의존하는 방법으로,
다른 아이템에 준 rating을 바탕으로, 특정 아이템에 줄 확률을 추정하여 이에 대한 기대값을 구하는 방식.

$$ r_{c,s} = E(r_{c,s}) = \sum_{i=0}^{n} i \times Pr(r_{c,s}=i | r_{c,s'})$$

이 값을 구하기 위해서는 확률을 추정해야 할 것이다.
확률 추정을 위해서는 cluster model이나 Bayesian network를 사용할 수 있다.

#### Latent factor models란?
- user-item에 평점이 기록된 matrix를 user-latent factor * latent factor-item 행렬로 나눈다.
- 이를 위해서는 Matrix Factorization이 사용된다.
- 주로 사용되는 factorization은 SVD. (후술)

행렬 분해를 하는 이유는, 특성 추출을 위한 것이다.
정확히는 User행렬과, feature 행렬을 쪼개서 특성 그 자체를 보기에 좋아질 것이라는 관점이다.

Remark. Explicit feedback & Implicit feedback
- Explicit feedback : 평점 데이터 등 user들이 item에 남긴 명백한 기록, 흥미들. 실저로 많은 평점을 남기지는 않을거기에 sparse matrix를 얻게 된다.
- Implicit feedback : 직접적인 의견으로는 연결되지 않는 유저의 행동(검색 이력, 마우스 이동 등), 이것이 반영될 경우 matrix가 densely-filled matrix를 얻게 된다.

### Limitations
물론 이 모델에도 한계점들은 존재한다.

#### New User Problem &  Item Problem
역시 content-based와 마찬가지로 새로운 유저가 들어왔을 때 추천을 위해서는, 시스템이 충분히 이 유저에 대해 학습할 기회가 주어져야 한다.

이는 아이템에 대해서도 적용되는데, 신규 아이템이 추천되기 위해서는 충분한 양의 평가가 이루어져야 한다. 그 전에는 새로운 아이템은 추천되기가 어려울 것이다.

#### Sparsity
추천 시스템의 근본적인 문제로 돌아오자.

추천 시스템은 기본적으로 굉장히 적은 양의 rating을 가지고 평가가 매겨지지 않은 item들의 rating을 추정 (extrapolization)해야 하는 문제를 풀어야 한다.

- 소수의 평가된 rating(example)들은 매우 중요하다.
- 소수가 평가될수록 아이템이 추천될 확률은 매우 적다.

이를 대처하는 방법은
- 인구 통계 기술 활용
- 행렬의 분해 활용 (또 **SVD**)

따라서, collaborative filtering을 논하기 위해서는 SVD를 논하지 않을 수가 없다.

### SVD의 개요
Matrix Factorization의 가장 기본적인 형태이다.
Latent Model에서 자주 사용된다.

#### SVD의 기본형태
주어진 $m \times n$ 형태의 matrix $A$에 대해
A를 다음과 같이 쪼개려는 것이다.

$$ A = U\Sigma V^{T}$$
여기서 각각의 요소는 다음과 같은 특징을 가지고 있다.
- $U$는 $AA^{T}$의 eigenvector들로 이루어진 Orthogonal한 matrix. 다시 말해 $UU^{T}=I$.
- $V$는 $A^{T}A$의 eigenvector들로 이루어진 Orthogonal한 matrix.
- $\Sigma$는 $A^{T}A$의 eigenvalue들의 square root. (이 값들을 특이값(singular value))
이 행렬의 특징은, 대각성분을 제외하고는 무조건 모두 0이라는 점이다.

몇 가지 확인해야 할 점(사실은 이미 알고 있어야 할)은.

- 대칭행렬은 고유값 분해가 가능하다.
- $AA^{T}$의 고유값은 무조건 양수이다.

#### Reduced SVD
위의 형태를 보면 다음과 같은 상황을 생각할 수 있다.

- $\Sigma$에서 대각 성분이 아닌 부분이 필요한가?  없앤다면 ->thin SVD
- $\Sigma$에서 singular value가 0인 부분을 없앤다면? -> compact SVD
- $\Sigma$에서 그 이상 없앤다면 -> Truncated SVD

위의 두개의 경우는 원래 A를 복구할 수 있으나 밑의 경우는 원래 A 대신 A를 근사시킨 행렬을 얻게 된다.


#### 추천시스템에서의 SVD의 의미
추천시스템에서 SVD를 적용하게 되면
$$ Rating = (user) * (singular\; val) * (item) $$
형태로 쪼갤 수 있다.

여기서 compact SVD를 적용한다면, singular val들은 특성들이 가지는 강도를 표현하게 될 것이다. 물론, 어떤 특성인지는 알 수 없지만.


## Hybrid Methods
Hybrid라는 말대로, 앞에서 나온 CB랑 CF를 모두 활용하는 방식이다.

[1]에 소개된 4가지 방식을 소개한다.

### Combining Separate Recommenders

이 방법의 경우는 CB, CF **각각에서 별도로 추천**을 받은 후
- 선형결합으로 결과를 변환
- 투표를 통해 결과를 선택
등의 방법으로 최종 추천을 실시한다.

혹은 여러개의 추천 시스템 중 "가장 성과가 좋은" 추천 시스템이 고른 추천을 제시하는 방법도 존재한다.



### Adding CB to CF
CF 기반의 모델에 CB를 섞는 이 방법은 기본적으로는 CF 기반으로 유저들 간의 유사성을 계산하나, CB 기반의 프로필을 유지하여, 프로필을 바탕으로 CF를 계산하기도 한다.

이 방법을 사용할 경우, CF에서 겪는 sparsity problem을 어느정도 극복할 수 있다. 실제로 두 유저들이 공통적으로 평점을 매긴 item은 정말 매우 극소량일것이기에.

### Adding CF to CB

이 경우는 CB에 MF, SVD등의 차원축소 기법을 활용한다.


### Developing a Single Unifying Recommendational model

두 방법을 혼합하여, 하나의 새로운 모델을 제시하는 방법이다.

#### Probabilistic Methods
CB, CF의 방법론을 확률적인 방법으로 섞어 사용하기도 한다.

[1]에서는 user특성과,  item특성을 바탕으로 특정 상수로 noise를 부과하여 rating을 추청하는 방법을 제시하고 있다.

#### Knowledge-based Techniques

이 경우는 domain 지식을 활용하여 추천 모델의 성능 향상을 꾀한다.

예를 들어, 음식 종류와 식성을 판단하여 채식주의자들에게 고기나 해산물들을 추천하지 않는 방법을 고려할 수 있을 것이다.

결론적으로 CB, CF보다는 Hybrid의 추천 결과가 더 좋다는 연구결과들이 존재한다.